<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2023/blob/main/Week12/CUDA_DynamicParallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-z2uhhe77
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-z2uhhe77
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=7f9a82b3d5894e2e8fe0a5adb2d35b5a73e7f669524307718d34e19fd5c17d34
  Stored in directory: /tmp/pip-ephem-wheel-cache-_nnvo0lz/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
%%writefile dp.cu
//example given here: https://stackoverflow.com/questions/64516177/call-kernel-inside-cuda-kernel

//this example wont work on Colab GPU since the required compute capability for 
//kernel launch from another kernel is compute capability 3.5
 
#include <stdio.h>
const int N = 100;//33 * 1024;
const int threadsPerBlock = 32;// 256;

#define imin(a,b) (a<b?a:b)

const int blocksPerGrid =  imin( 32, (N+threadsPerBlock-1) / threadsPerBlock );

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

__global__ void kernel( float *a, float *b, int N ) 
{
	int i = threadIdx.x + blockDim.x*blockIdx.x;
	if(i < N)
	{
			for(int j=i; j< min(i+5, N); j++)
			{
					b[i] += a[j];
			}
	}
}

__global__ void kernel_child(int start, int end, float *a, float *b) 
{
	int j = start + threadIdx.x + blockDim.x*blockIdx.x;
	if(j < end)
	{
			b[j] += a[j];			
	}
}

__global__ void kernel_parent( float *a, float *b, int start, int end ) 
{
	int i = threadIdx.x + blockDim.x*blockIdx.x;

	b[i] = a[i];	
	kernel_child<<< (end-start)/32, 32>>>(start, end, a, b);			 
}



int main() 
{ 
	float   *a = 0; 
  float   *b = 0;
	float   *c = 0;

	// Allocate Unified Memory -- accessible from CPU or GPU
	checkCudaErr(cudaMallocManaged(&a, N*sizeof(float)), "cudaMallocManaged1");
	checkCudaErr(cudaMallocManaged(&b, N*sizeof(float)), "cudaMallocManaged2"); 
	checkCudaErr(cudaMallocManaged(&c, N*sizeof(float)), "cudaMallocManaged3"); 

	// fill in the memory with data
	for (int i=0; i<N; i++) {
		a[i] = i+1; 
		b[i] = 0;
		c[i] = 0;
	} 

	kernel<<<blocksPerGrid,threadsPerBlock>>>( a, b, N );

	cudaDeviceSynchronize();

	kernel_parent<<<blocksPerGrid,threadsPerBlock>>>( a, b, 0, N-1 );
  
	for(int i=0; i<N;++i)
		printf("%f\n", b[i]);
	
	// free memory on the gpu side
	checkCudaErr( cudaFree( a ) , "cudaFree1");
	checkCudaErr( cudaFree( b ) , "cudaFree2"); 
	checkCudaErr( cudaFree( c ) , "cudaFree3"); 
	checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

	return 0;
}

/tmp/tmp21ykqr3p/4329d123-c15b-4f72-b8d7-c52c0c634ad8.cu(46): error: kernel launch from __device__ or __global__ functions requires separate compilation mode

1 error detected in the compilation of "/tmp/tmp21ykqr3p/4329d123-c15b-4f72-b8d7-c52c0c634ad8.cu".



In [ ]:
!nvcc -rdc=true dp.cu -o dp
!./dp